In [1]:
!pip install -q tabula-py
!pip install PyMuPDF
!pip install selenium requests beautifulsoup4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.7/475.7 kB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.9 MB/s eta 0:00:00


In [2]:
import os
import re
import tabula
import pandas as pd
import numpy as np
import fitz  # module from PyMuPDF

In [3]:
# Set display options to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [10]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
import os
import re
import time

# Function to convert month string to mm format
def month_to_mm(month_str):
    return {
        'January': '01', 'February': '02', 'March': '03', 'April': '04',
        'May': '05', 'June': '06', 'July': '07', 'August': '08',
        'September': '09', 'October': '10', 'November': '11', 'December': '12'
    }.get(month_str, '')

options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run headless Chrome (no GUI)
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-gpu')
options.add_argument('--window-size=1920x1080')

driver = webdriver.Chrome(options=options)

try:
    driver.get("https://company.meralco.com.ph/news-and-advisories/rates-archives")

    # Ask user for month and year
    month = input("Enter the month (e.g., January): ").strip().title()
    year = input("Enter the year (e.g., 2023): ").strip()
    month_mm = month_to_mm(month)

    # Click "Show more" until the desired date is found or no more items are available
    date_found = False
    while not date_found:
        try:
            show_more_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//a[@class='btn btn-bordered load-more' and @title='Go to next page']"))
            )
            show_more_button.click()
            time.sleep(2)  # Wait for content to load

            rows = driver.find_elements(By.XPATH, "//tr[.//span[contains(text(), 'Summary Schedule of Rates') or contains(text(), 'Summary of Schedule of Rates')]]")

            for row in rows:
                try:
                    # Find the date in the same row
                    date_element = row.find_element(By.XPATH, ".//td[contains(@class, 'views-field-field-date-created')]")
                    date_text = date_element.text.strip()

                    # Convert date text to "mm-yy" format
                    match = re.search(r'(\w+)\s+(\d{4})', date_text)
                    if match:
                        month_str, year_str = match.groups()
                        if month_str == month and year_str == year:
                            # Find the PDF link
                            pdf_link_element = row.find_element(By.XPATH, ".//a[contains(@class, 'btn-bordered-orange') and contains(@href, '.pdf')]")
                            pdf_link = pdf_link_element.get_attribute('href')
                            date_found = True  # Set flag to true if date is found
                            break

                    else:
                        print(f"Date format not recognized for text: {date_text}")
                        continue  # Skip this row if date format is not recognized

                except Exception as e:
                    print(f"Error processing row: {e}")

        except Exception as e:
            print("No more items available or error clicking 'Show more'")
            break  # Break the loop if no more "Show more" button is found or error occurs

    # Download the PDF if a link was found
    if pdf_link:
        response = requests.get(pdf_link)
        if response.status_code == 200:
            pdf_name = f"Summary_Schedule_of_Rates_{month_mm}-{year[-2:]}.pdf"
            with open(f'{pdf_name}', 'wb') as f:
                f.write(response.content)
            print(f"Successfully downloaded PDF from {pdf_link}")
        else:
            print(f"Failed to download PDF from {pdf_link}, status code: {response.status_code}")
    else:
        print(f"No PDF found for {month} {year}")

finally:
    driver.quit()


Enter the month (e.g., January): February
Enter the year (e.g., 2023): 2023
Successfully downloaded PDF from https://meralcomain.s3.ap-southeast-1.amazonaws.com/2023-02/02-2023_rate_schedule.pdf


In [91]:
headers_and_subheaders = {
    "Customer Subclass": "NaN",
    "Generation Charge": "per kWh",
    "Prev. Mos' Adj on Gen": "per kWh",
    "Power Act Reduction": "per kWh",
    "Transmission Charge": "per kWh, per kW",
    "System Loss Charge": "per kWh",
    "Distribution Charge": "per kWh, per kW",
    "Supply Charge": "per kWh, per cust/mo",
    "Metering Charge": "per kWh, per cust/mo",
    "Reset Cost Adjustment": "per kWh",
    "Distribution Rate True-Up": "per kWh",
    "Dist Rate True-Up 1": "per kWh",
    "Dist Rate True-Up 2": "per kWh",
    "Dist Rate True-Up 3": "per kWh",
    "Dist Rate True-Up 4": "per kWh",
    "For Non-lifeline: Lifeline Subsidy": "per kWh",
    "For Lifeline: Applicable Discounts": "discount %",
    "Lifeline Rate Subsidy": "per kWh",
    "Senior Citizen Subsidy": "per kWh",
    "Current RPT Charge": "per kWh",
    "UC-ME":"per kWh",
    "UC-ME for (NPC-SPUG)": "per kWh",
    "UC-ME for (RED-CI)": "per kWh",
    "UC-EC": "per kWh",
    "UC-SCC": "per kWh",
    "UC-SD": "per kWh",
    "Fit-All (Renewable)": "per kWh",
    "Cross Subsidy Charge": "per kWh",
    "Current RPT": "per kWh",
    "Lifeline Discount": "%",
    "Special Discount": "%",
    "Power Factor Adj": "Penalty, Disc",
    "Prev. Mos' Adj on Gen Cost": "per kWh"
}

In [107]:
date_part = pdf_name.split('_')[-1].replace('.pdf', '')

df = tabula.read_pdf(pdf_name, lattice=True, pages=1)[0]

In [108]:
def clean_column_name(col):
  col = col.replace('\r', ' ')  # Replace '\r' with a space
  col = re.sub(r'\s+', ' ', col)  # Replace multiple spaces with a single space
  # Special handling for "Dist Rate True-Up"
  if any(rate in col for rate in ['Dist Rate True-Up', 'Distribution Rate True-Up']):
      # Replace only '\r' and extra spaces, keep digits
      col = re.sub(r'[^\w\s\d-]', '', col)

  else:
      col = re.sub(r'\s*\d+\s*', ' ', col) # Remove superscripts

  return col.strip()  # Strip leading/trailing whitespace

In [109]:
def count_true_up_occurrences(columns):
    true_up_counter = 0
    for col in columns:
        if 'Up' in col:
            true_up_counter += 1
    return true_up_counter

In [110]:
def convert_column_names(columns):
    true_up_count = count_true_up_occurrences(columns)
    renamed_columns = []

    for col in columns:
        if 'Up' in col:
            if true_up_count == 1:
                renamed_columns.append("Distribution Rate True-Up")
            else:
                match = re.search(r'True-\s*Up\s*(\d)', col)
                if match:
                    num = match.group(1)
                    renamed_columns.append(f"Dist Rate True-Up {num}")
        else:
            renamed_columns.append(col)

    return renamed_columns

In [111]:
def process_headers(df):
  df.columns = convert_column_names(df.columns)
  df.columns = [clean_column_name(col) for col in df.columns]

  # Ensure unique column names by appending the DataFrame index if needed
  df.columns = [f"{col}_{i}" if col in df.columns[:i] else col for i, col in enumerate(df.columns)]

  # Clean the first row in a similar way and assign it back to the DataFrame
  df.iloc[0] = [clean_column_name(str(value)) for value in df.iloc[0]]

  non_null_values = df.iloc[0].dropna().values
  non_null_values = [val for val in non_null_values if val != 'nan']

  if "Universal Charge" in df.columns:
      universal_charge_index = df.columns.get_loc("Universal Charge")

      # Replace "Universal Charge" with the first non-null value
      df.columns.values[universal_charge_index] = non_null_values[0]

      # Insert any additional non-null values into the column headers
      remaining_values = non_null_values[1:]

      if remaining_values:
          # Split the columns into parts: before, at, and after "Universal Charge"
          before = df.columns[:universal_charge_index + 1]
          after = df.columns[universal_charge_index + 1:]

          # Create new columns with the remaining values inserted after the "Universal Charge" replacement
          new_columns = list(before) + remaining_values + list(after[:-len(remaining_values)])
          df.columns = new_columns

  # Replace "Lifeline Eligibility" with "For Non-lifeline: Lifeline Subsidy" and insert "For Lifeline: Applicable Discounts"
  if "Lifeline Eligibility" in df.columns:
      lifeline_index = df.columns.get_loc("Lifeline Eligibility")
      df.columns.values[lifeline_index] = "For Non-lifeline: Lifeline Subsidy"

      df.columns = list(df.columns[:lifeline_index + 1]) + ["For Lifeline: Applicable Discounts"] + list(df.columns[lifeline_index + 1:-1])

  # Replace column containing "SUMMARY" with "Customer Subclass"
  summary_column = next((col for col in df.columns if "SUMMARY" in col), None)
  if summary_column:
      df.rename(columns={summary_column: "Customer Subclass"}, inplace=True)

  # Prepare lists for multiindex tuples
  multi_index_headers = []

  for col in df.columns:
    if col in headers_and_subheaders:
        subheader = headers_and_subheaders[col].split(", ")
        for sub in subheader:
            multi_index_headers.append((col, sub))

  multi_index = pd.MultiIndex.from_tuples(multi_index_headers, names=['Charge', 'Unit'])

  df.columns = multi_index

  return df

In [112]:
def crop_dataframe(df):
  start_index = df[df.iloc[:, 0] == 'Residential'].index[0]
  end_index = df[df.iloc[:, 0] == '400 W HPS (or equivalent)'].index[0]

  df = df.iloc[start_index+1:end_index + 1]
  df.reset_index(drop=True, inplace=True)

  return df

In [113]:
df = crop_dataframe(process_headers(df))

In [114]:
df['Date'] = date_part

In [115]:
def is_valid_float(value):
  try:
      return not pd.isna(value) and float(value) is not None
  except ValueError:
      return False

is_valid_float_df = df.applymap(is_valid_float)

# Keep rows where there is at least one valid float-representing string
df = df[is_valid_float_df.any(axis=1)]

In [116]:
def clean_subclass(subclass):
    if isinstance(subclass, str):
        if subclass.endswith("14"):
            subclass = subclass[:-2]

        if "HPS" in subclass:
            subclass = subclass.replace(" (or equivalent)", "(or equivalent)")

        if subclass == "VERY LARGE AND EXTRA LARGE 115 KV/69 K":
            subclass = "VERY LARGE AND EXTRA LARGE 115 KV/69 KV"

    return subclass

df[("Customer Subclass", "NaN")] = df[("Customer Subclass", "NaN")].apply(clean_subclass)

In [117]:
def map_customer_class(df):
    class_mapping = {
        "Residential": ["0 TO 20 KWH", "21 TO 50 KWH", "51 TO 70 KWH",
                        "71 TO 100 KWH", "101 TO 200 KWH", "201 TO 300 KWH",
                        "301 TO 400 KWH", "OVER 400 KWH"],
        "General Service A": ["0 TO 200 KWH", "201 TO 300 KWH", "301 TO 400 KWH", "OVER 400 KWH"],
        "General Service B": ["General Service B"],
        "General Power (GP) Secondary": ["MEDIUM SECONDARY", "LARGE SECONDARY", "VERY LARGE SECONDARY"],
        "GP 13.8 KV and below": ["MEDIUM 13.8 KV AND BELOW", "LARGE 13.8 KV AND BELOW", "VERY LARGE 13.8 KV AND BELOW"],
        "GP 34.5 KV": ["MEDIUM 34.5 KV", "LARGE 34.5 KV", "VERY LARGE AND EXTRA LARGE 34.5 KV"],
        "GP 115 KV / 69 KV": ["LARGE 115 KV/69 KV", "VERY LARGE AND EXTRA LARGE 115 KV/69 KV"],
        "GHMSCI": ["GHMSCI"]
    }

    customer_class = []
    lower_limit_demand = []
    upper_limit_demand = []
    residential_flag = True
    general_service_a_flag = False
    flat_streetlights_flag = False

    for subclass in df[("Customer Subclass", "NaN")]:
        assigned_class = None
        lower_limit = None
        upper_limit = None

        if flat_streetlights_flag:
            assigned_class = "FLAT STREETLIGHTS"
        elif subclass == "Per kWh" or subclass == "125 W Mercury, 70 W HPS(or equivalent)":
            assigned_class = "FLAT STREETLIGHTS"
            flat_streetlights_flag = True
        elif residential_flag:
            assigned_class = "Residential"
            if subclass == "OVER 400 KWH":
                residential_flag = False
                general_service_a_flag = True
        elif general_service_a_flag:
            assigned_class = "General Service A"
            if subclass == "OVER 400 KWH":
                general_service_a_flag = False
        else:
            for class_name, subclasses in class_mapping.items():
                if subclass in subclasses:
                    assigned_class = class_name
                    break

        # Set the lower and upper limit demand based on the subclass
        if subclass == "General Service B":
            lower_limit = 5
            upper_limit = 40
        elif "MEDIUM" in subclass:
            lower_limit = 40
            upper_limit = 200
        elif "LARGE" in subclass:
            lower_limit = 200
            upper_limit = 750
        elif "VERY LARGE" in subclass:
            lower_limit = 750
            upper_limit = 10000
        elif "EXTRA LARGE" in subclass:
            lower_limit = 10000
            upper_limit = np.nan

        customer_class.append(assigned_class if assigned_class else "Unknown")
        lower_limit_demand.append(lower_limit)
        upper_limit_demand.append(upper_limit)

    df["Customer Class"] = customer_class
    df["Lower Limit Demand"] = lower_limit_demand
    df["Upper Limit Demand"] = upper_limit_demand
    return df

# Apply the mapping function
df = map_customer_class(df)

# Remove the Date column
#df = df.drop(('Date', ''), axis=1)

# Reorder the columns
#cols = [("Customer Class", ""), ("Customer Subclass", "NaN"), ("Lower Limit Demand", ""), ("Upper Limit Demand", "")] + [col for col in df.columns if col not in [("Customer Class", ""), ("Customer

df

Charge                        Customer Subclass Generation Charge  \
Unit                                        NaN           per kWh   
0                                   0 TO 20 KWH            6.9154   
1                                  21 TO 50 KWH            6.9154   
2                                  51 TO 70 KWH            6.9154   
3                                 71 TO 100 KWH            6.9154   
4                                101 TO 200 KWH            6.9154   
5                                201 TO 300 KWH            6.9154   
6                                301 TO 400 KWH            6.9154   
7                                  OVER 400 KWH            6.9154   
9                                  0 TO 200 KWH            6.9154   
10                               201 TO 300 KWH            6.9154   
11                               301 TO 400 KWH            6.9154   
12                                 OVER 400 KWH            6.9154   
13                            General Service B            6.9154   
15                             MEDIUM SECONDARY            6.9154   
16                              LARGE SECONDARY            6.9154   
17                         VERY LARGE SECONDARY            6.9154   
19                     MEDIUM 13.8 KV AND BELOW            6.9154   
20                      LARGE 13.8 KV AND BELOW            6.9154   
21                 VERY LARGE 13.8 KV AND BELOW            6.9154   
23                               MEDIUM 34.5 KV            6.9154   
24                                LARGE 34.5 KV            6.9154   
25           VERY LARGE AND EXTRA LARGE 34.5 KV            6.9154   
27                           LARGE 115 KV/69 KV            6.9154   
28      VERY LARGE AND EXTRA LARGE 115 KV/69 KV            6.9154   
29                                       GHMSCI            6.9154   
31                                      Per kWh            6.9154   
33       125 W Mercury, 70 W HPS(or equivalent)            290.45   
34      250 W Mercury, 150 W HPS(or equivalent)            463.33   
35      400 W Mercury, 250 W HPS(or equivalent)            774.52   
36                     400 W HPS(or equivalent)          1,203.28   

Charge Transmission Charge           System Loss Charge Distribution Charge  \
Unit               per kWh    per kW            per kWh             per kWh   
0                   0.8322       NaN             0.6223              0.9803   
1                   0.8322       NaN             0.6223              0.9803   
2                   0.8322       NaN             0.6223              0.9803   
3                   0.8322       NaN             0.6223              0.9803   
4                   0.8322       NaN             0.6223              0.9803   
5                   0.8322       NaN             0.6223              1.2908   
6                   0.8322       NaN             0.6223              1.5837   
7                   0.8322       NaN             0.6223              2.0941   
9                   0.8322       NaN             0.6223              0.9803   
10                  0.8322       NaN             0.6223              1.2908   
11                  0.8322       NaN             0.6223              1.5837   
12                  0.8322       NaN             0.6223              2.0941   
13                     NaN  266.4100             0.6223              0.1339   
15                     NaN  290.6500             0.6223              0.1339   
16                     NaN  290.6500             0.6223              0.1339   
17                     NaN  290.6500             0.6223              0.1339   
19                     NaN  304.0900             0.2483              0.0502   
20                     NaN  304.0900             0.2483              0.0502   
21                     NaN  304.0900             0.2483              0.0502   
23                     NaN  336.4800             0.2483              0.0502   
24                     NaN  336.4800             0.24

In [118]:
def convert_value(val):
    if pd.isna(val) or val == "nan":
        return np.nan
    if isinstance(val, str):
        if val.startswith('(') and val.endswith(')'):
            return -float(val[1:-1])
        if val.endswith('%'):
            return float(val.strip('%')) / 100
        try:
            return float(val)
        except ValueError:
            return val  # Leave non-numeric strings as is
    return val

numeric_df = df.applymap(convert_value)
numeric_df = numeric_df.apply(pd.to_numeric, errors='coerce')

exclude_cols_kwh = ['Transmission Charge', 'Distribution Charge', 'Customer Class', 'Customer Subclass']
exclude_cols_kw = exclude_cols_kwh

combined_kwh = numeric_df.loc[:, numeric_df.columns.get_level_values(1) == 'per kWh'].drop(columns=exclude_cols_kwh, level=0).sum(axis=1, skipna=True)
combined_kw = numeric_df.loc[:, numeric_df.columns.get_level_values(1) == 'per kW'].drop(columns=exclude_cols_kw, level=0).sum(axis=1)

df[('kWh Attributable', np.nan)] = combined_kwh
df[('kW Attributable', np.nan)] = combined_kw

In [119]:
df

Charge                        Customer Subclass Generation Charge  \
Unit                                        NaN           per kWh   
0                                   0 TO 20 KWH            6.9154   
1                                  21 TO 50 KWH            6.9154   
2                                  51 TO 70 KWH            6.9154   
3                                 71 TO 100 KWH            6.9154   
4                                101 TO 200 KWH            6.9154   
5                                201 TO 300 KWH            6.9154   
6                                301 TO 400 KWH            6.9154   
7                                  OVER 400 KWH            6.9154   
9                                  0 TO 200 KWH            6.9154   
10                               201 TO 300 KWH            6.9154   
11                               301 TO 400 KWH            6.9154   
12                                 OVER 400 KWH            6.9154   
13                            General Service B            6.9154   
15                             MEDIUM SECONDARY            6.9154   
16                              LARGE SECONDARY            6.9154   
17                         VERY LARGE SECONDARY            6.9154   
19                     MEDIUM 13.8 KV AND BELOW            6.9154   
20                      LARGE 13.8 KV AND BELOW            6.9154   
21                 VERY LARGE 13.8 KV AND BELOW            6.9154   
23                               MEDIUM 34.5 KV            6.9154   
24                                LARGE 34.5 KV            6.9154   
25           VERY LARGE AND EXTRA LARGE 34.5 KV            6.9154   
27                           LARGE 115 KV/69 KV            6.9154   
28      VERY LARGE AND EXTRA LARGE 115 KV/69 KV            6.9154   
29                                       GHMSCI            6.9154   
31                                      Per kWh            6.9154   
33       125 W Mercury, 70 W HPS(or equivalent)            290.45   
34      250 W Mercury, 150 W HPS(or equivalent)            463.33   
35      400 W Mercury, 250 W HPS(or equivalent)            774.52   
36                     400 W HPS(or equivalent)          1,203.28   

Charge Transmission Charge           System Loss Charge Distribution Charge  \
Unit               per kWh    per kW            per kWh             per kWh   
0                   0.8322       NaN             0.6223              0.9803   
1                   0.8322       NaN             0.6223              0.9803   
2                   0.8322       NaN             0.6223              0.9803   
3                   0.8322       NaN             0.6223              0.9803   
4                   0.8322       NaN             0.6223              0.9803   
5                   0.8322       NaN             0.6223              1.2908   
6                   0.8322       NaN             0.6223              1.5837   
7                   0.8322       NaN             0.6223              2.0941   
9                   0.8322       NaN             0.6223              0.9803   
10                  0.8322       NaN             0.6223              1.2908   
11                  0.8322       NaN             0.6223              1.5837   
12                  0.8322       NaN             0.6223              2.0941   
13                     NaN  266.4100             0.6223              0.1339   
15                     NaN  290.6500             0.6223              0.1339   
16                     NaN  290.6500             0.6223              0.1339   
17                     NaN  290.6500             0.6223              0.1339   
19                     NaN  304.0900             0.2483              0.0502   
20                     NaN  304.0900             0.2483              0.0502   
21                     NaN  304.0900             0.2483              0.0502   
23                     NaN  336.4800             0.2483              0.0502   
24                     NaN  336.4800             0.24